# Readme
*Please keep this code confidential to included members.*

Features needed<br>
* ROI selection, approx. 100 neurons.
* Time series extraction (SNR df/F; F ~ baseline signal; df = f_t - F).
* Masking (cell identification)
    1. Use degree centrality
    2. Sum, threshold
* Cross-MI, cross-correlation matrices
* Lag (source vs. reference onset order)
* Automation



# Initialize program environment
Run this first.

## Load libraries

In [3]:
#! pip install cupy
#! pip install hdf5storage
! pip install ipympl
! pip install ipywidgets
! pip install opencv-python
! pip install plotly
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import math
import numpy
import cupy
import sys
import scipy.io
import os
import h5py
#import hdf5storage
import warnings
import time
import gc
from numpy import matlib as matlib
from numba import jit, cuda
from ipywidgets import widgets, Label, Layout, HBox, VBox
from IPython.display import clear_output, display
from skimage import io
from IPython.display import HTML
import csv

warnings.filterwarnings("ignore")
gc.collect()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Mutual information

In [ ]:
# Gaussian kernel density estimation function
def gkde_mi_gpu(W,V,r,lag,MI,R,pdfX,pdfY,pdfXY,series_length):

  b = -1

  for tlag in lag:
    # Preliminary parameters
    b   = b + 1
    X   = V[:,tlag:series_length - lag[-1] + tlag]
    Y   = W[:,0:series_length - lag[-1]]
    Y   = W[:,0:series_length - lag[-1]]
    XY  = cupy.array([X,Y]) # Currently shape is 2,X.shape[0],X.shape[1]
    n   = X.shape[1]
    d   = 1
    dxy = 2
    h   = ((4 / (d + 2)) ** (1 / (d + 4))*(n ** ((- 1) / (d + 4))))
    hxy = ((4 / (dxy + 2)) ** (1 / (dxy + 4))*(n ** ((- 1) / (dxy + 4))))
    rX  = cupy.reshape((cupy.amax(X,axis=1) - cupy.amin(X,axis=1)) / r,(-1,1))
    rY  = cupy.reshape((cupy.amax(Y,axis=1) - cupy.amin(Y,axis=1)) / r,(-1,1))
    dX  = (cupy.reshape((cupy.amax(X,axis=1) - cupy.amin(X,axis=1)),(-1,1)) - rX) / rX
    dY  = (cupy.reshape((cupy.amax(Y,axis=1) - cupy.amin(Y,axis=1)),(-1,1)) - rY) / rY
    uX  = cupy.reshape(cupy.amin(X,axis=1),(-1,1)) + rX / 2 + cupy.repeat(cupy.reshape(cupy.arange(0,dX[0]),(1,-1)),rX.shape[0],axis=0)*rX
    uY  = cupy.reshape(cupy.amin(Y,axis=1),(-1,1)) + rY / 2 + cupy.repeat(cupy.reshape(cupy.arange(0,dY[0]),(1,-1)),rY.shape[0],axis=0)*rY
    
    if uX.shape[1] == (r - 1):
      uX = cupy.concatenate((uX,cupy.reshape(cupy.amax(X,axis=1),(-1,1)) - rX / 2),axis=1)
    if uY.shape[1] == (r - 1):
      uY = cupy.concatenate((uY,cupy.reshape(cupy.amax(Y,axis=1),(-1,1)) - rY / 2),axis=1)

    uXY    = cupy.array([uX,uY]) # Currently shape is 2,X.shape[0],X.shape[1]
    EX     = cupy.reshape(cupy.sum(X,axis=1) / X.shape[1],(-1,1))
    EY     = cupy.reshape(cupy.sum(Y,axis=1) / Y.shape[1],(-1,1))
    SX     = cupy.reshape(cupy.sum((X - EX) ** 2,axis=1) / (X.shape[1] - 1),(-1,1))
    SY     = cupy.reshape(cupy.sum((Y - EY) ** 2,axis=1) / (Y.shape[1] - 1),(-1,1))
    sXY    = cupy.reshape(cupy.sum((X - EX)*(Y - EY),axis=1) / (X.shape[1] - 1),(-1,1))
    SXY    = cupy.array([cupy.concatenate((SX,sXY),axis=1),cupy.concatenate((sXY,SY),axis=1)]) # Currently shape is 2,X.shape[0],2
    detSXY = cupy.reshape((SXY[0,:,0]*SXY[1,:,1]) - (SXY[0,:,1]*SXY[1,:,0]),(-1,1))
    C      = cupy.array([cupy.concatenate((cupy.reshape(SXY[1,:,1],(-1,1)),cupy.reshape(-SXY[0,:,1],(-1,1))),axis=1),cupy.concatenate((cupy.reshape(-SXY[1,:,0],(-1,1)),cupy.reshape(SXY[0,:,0],(-1,1))),axis=1)])
    iSXY   = C*1/detSXY

    # Set PDF place holders
    PX  = cupy.zeros((X.shape[0],r))
    PY  = cupy.zeros((Y.shape[0],r))
    PXY = cupy.zeros((X.shape[0],r,r))

    for i in cupy.arange(0,r):
      uXX     = ((cupy.reshape(uX[:,i],(-1,1))-X)**2)*(SX**-1)/(h**2)
      KX      = cupy.exp(-uXX/2)/((2*cupy.pi)**(d/2)*(h**d)*(SX**0.5))
      PX[:,i] = cupy.sum(KX/n*rX,axis=1)
      uYY     = ((cupy.reshape(uY[:,i],(-1,1))-Y)**2)*(SY**-1)/(h**2)
      KY      = cupy.exp(-uYY/2)/((2*cupy.pi)**(d/2)*(h**d)*(SY**0.5))
      PY[:,i] = cupy.sum(KY/n*rY,axis=1)

      for j in cupy.arange(0,r):
          duXY       = cupy.reshape(cupy.array([uXY[0,:,i],uXY[1,:,j]]),(2,uXY.shape[1],1)) - XY
          uuXYXY     = cupy.array([cupy.sum(duXY*cupy.reshape(iSXY[:,:,0],(2,iSXY.shape[1],1)),axis=0),cupy.sum(duXY*cupy.reshape(iSXY[:,:,1],(2,iSXY.shape[1],1)),axis=0)])
          uXYXY      = cupy.sum(uuXYXY*duXY/(hxy**2),axis=0)
          KXY        = cupy.exp(-uXYXY/2)/((2*cupy.pi)**(dxy/2)*(hxy**dxy)*(detSXY**0.5))
          PXY[:,i,j] = cupy.sum(KXY/n*rX*rY,axis=1)

    PX  = PX/cupy.reshape(cupy.sum(PX,axis=1),(-1,1))
    PY  = PY/cupy.reshape(cupy.sum(PY,axis=1),(-1,1))
    PXY = PXY/cupy.reshape(cupy.sum(cupy.sum(PXY,axis=2),axis=1),(-1,1,1))
    PXY[cupy.isnan(PXY)] = 0
    pdfX[:,:,b]    = PX
    pdfY[:,:,b]    = PY
    pdfXY[:,:,:,b] = PXY

    for i in cupy.arange(0,r):
      logPXY = cupy.log2(PXY[:,i,:] / cupy.reshape(PX[:,i],(-1,1)) / PY)
      logPXY[PXY[:,i,:]<=10**-8] = 0
      MI[:,b] = MI[:,b] + cupy.sum(PXY[:,i,:]*logPXY,axis=1)

    # Compute R (independent of bins) per lag
    R[:,b] = SXY[1,:,0] / cupy.reshape(cupy.sqrt(SX)*cupy.sqrt(SY),(-1))

  return MI,R,pdfX,pdfY,pdfXY

def mutualinfo_gpu(W,V,r,lag,nshuf):
  series_length = W.shape[1]
  MI            = cupy.zeros((W.shape[0],lag.size))
  R             = cupy.zeros((W.shape[0],lag.size))
  pdfX          = cupy.zeros((W.shape[0],r,lag.size))
  pdfY          = cupy.zeros((W.shape[0],r,lag.size))
  pdfXY         = cupy.zeros((W.shape[0],r,r,lag.size))

  # COMPUTE MI
  MI,R,pdfX,pdfY,pdfXY = gkde_mi_gpu(W,V,r,lag,MI,R,pdfX,pdfY,pdfXY,series_length)

  # MOVE TO HOST
  pdfX  = cupy.asnumpy(pdfX)
  pdfY  = cupy.asnumpy(pdfY)
  pdfXY = cupy.asnumpy(pdfXY)
  PDF   = {'X':pdfX,'Y':pdfY,'XY':pdfXY}

  # SHUFFLING MI COMPARISON
  PMI    = cupy.zeros((W.shape[0],lag.size))
  PR     = cupy.zeros((W.shape[0],lag.size))
  if nshuf > 0:
    sW     = W.copy()
    slag   = cupy.arange(0,1)
    tMI    = cupy.zeros((W.shape[0],slag.size))
    tR     = cupy.zeros((W.shape[0],slag.size))
    tpdfX  = cupy.zeros((W.shape[0],r,slag.size))
    tpdfY  = cupy.zeros((W.shape[0],r,slag.size))
    tpdfXY = cupy.zeros((W.shape[0],r,r,slag.size))
    sMI    = cupy.zeros((W.shape[0],nshuf))
    sR     = cupy.zeros((W.shape[0],nshuf))

    for s in cupy.arange(0,nshuf):
      cupy.random.shuffle(cupy.transpose(sW))
      smi,sr,_,_,_ = gkde_mi_gpu(sW,V,r,slag,tMI,tR,tpdfX,tpdfY,tpdfXY,series_length)
      sMI[:,s] = cupy.reshape(smi,(-1))
      sR[:,s]  = cupy.reshape(sr,(-1))

    # COMPUTE p(MI > sMI) AND p(R ~= sR)
    for t in cupy.arange(0,lag.size):
      PMI[:,t] = 1 - cupy.sum(cupy.reshape(MI[:,t],(-1,1)) > sMI,axis=1)/nshuf

      nsR = sR.copy()
      psR = sR.copy()
      cupy.place(nsR,cupy.repeat(cupy.reshape(R[:,t]>0,(-1,1)),nshuf,axis=1),-1)
      cupy.place(psR,cupy.repeat(cupy.reshape(R[:,t]<0,(-1,1)),nshuf,axis=1),1)
      PR[:,t]  = 1 - (cupy.sum(cupy.reshape(R[:,t],(-1,1)) < nsR,axis=1) + cupy.sum(cupy.reshape(R[:,t],(-1,1)) > psR,axis=1))/nshuf

  # MOVE TO HOST
  MI  = cupy.asnumpy(MI)
  R   = cupy.asnumpy(R)
  PMI = cupy.asnumpy(PMI)
  PR  = cupy.asnumpy(PR)

  return MI,R,PDF,PMI,PR

# Compute voxel batches for parallelization
def batcher(nb,nvox):
    
  bsize = math.ceil(nvox/nb)
  blen = math.ceil(nvox/bsize) 
  mp = []
  for bn in numpy.arange(0,blen):
    mp.append(numpy.arange(bn*bsize,bn*bsize+bsize))
  mp[-1] = numpy.arange(mp[-1][0],nvox)

  return mp

# Run MI function from button
def run_mi(x,y,r,lag,nshuf,nbatch):

  # BEGIN
  s = time.time()

  d = cupy.asarray(buf[:,y,x,0]) # source series

  # Batching
  nvox  = D.shape[0]
  mp    = batcher(nbatch,nvox)

  # Initialize
  MI = np.zeros((nvox,lag.shape[0]))
  R  = np.zeros((nvox,lag.shape[0]))

  # Process
  for i in np.arange(len(mp)):
    print('Processing batch ' + str(i+1) + '/' + str(len(mp)) + ': bin size ' + str(r) + ' lag ' + str(lag[0]) + '-' + str(lag[-1]) + ' shuffles ' + str(nshuf))
    W = cupy.repeat(cupy.reshape(d,(1,-1)),mp[i].shape[0],axis=0)
    Y = cupy.asarray(D[mp[i],:]) # reference series
    MI[mp[i]],R[mp[i]],_,_,_ = mutualinfo_gpu(W,Y.copy(),r,lag,nshuf)
  
  print('Done in ',str(time.time()-s),' sec!')

  return MI,R

## Degree centrality

In [66]:
# Read data (flat code)
#cap = cv2.VideoCapture('/home/joshgoh/Desktop/Organoid/Data/source_data/ses-02/3.avi')
cap = cv2.VideoCapture('/content/drive/Shareddrives/Organoid/Data/source_data/test_ses/zoom 100x100.avi')
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))
fc = 0
ret = True

while (fc < frameCount  and ret):
  ret, buf[fc] = cap.read()
  fc += 1

cap.release()
D = buf[:,:,:,0]
#D = np.reshape(D,(D.shape[0],D.shape[1]*D.shape[2]))
#img = buf[:, :, :, 0]

In [ ]:
# Toy data D.
tc = np.random.rand(1,500)
D = np.random.rand(500,100,100)
D[:,0,0] = tc
for i in np.arange(2,2+20):
    for j in np.arange(2,2+20):
        D[:,i,j] = 0 + 0.5*tc + 0.5*np.random.rand(1,500)

In [67]:
# Show D
fig_D_heatmap = px.imshow(D[3,:,:])
fig_D_heatmap.show()
fig_D_lines = go.Figure()
fig_D_lines.add_trace(go.Scatter(x = np.arange(100), y = D[:,0,0], mode = 'lines', name = '0,0'))
fig_D_lines.add_trace(go.Scatter(x = np.arange(100), y = D[:,2,2], mode = 'lines', name = '2,2'))
fig_D_lines.add_trace(go.Scatter(x = np.arange(100), y = D[:,2,3], mode = 'lines', name = '2,3'))
fig_D_lines.add_trace(go.Scatter(x = np.arange(100), y = D[:,3,2], mode = 'lines', name = '3,2'))
fig_D_lines.add_trace(go.Scatter(x = np.arange(100), y = D[:,3,3], mode = 'lines', name = '3,3'))
fig_D_lines.show()
Dr = np.reshape(D,(500,100*100))
fig_D_lines_reshape = go.Figure()
fig_D_lines_reshape.add_trace(go.Scatter(x = np.arange(100), y = Dr[:,0], mode = 'lines', name = '0,0'))
fig_D_lines_reshape.add_trace(go.Scatter(x = np.arange(100), y = Dr[:,202], mode = 'lines', name = '2,2'))
fig_D_lines_reshape.add_trace(go.Scatter(x = np.arange(100), y = Dr[:,203], mode = 'lines', name = '2,3'))
fig_D_lines_reshape.add_trace(go.Scatter(x = np.arange(100), y = Dr[:,302], mode = 'lines', name = '3,2'))
fig_D_lines_reshape.add_trace(go.Scatter(x = np.arange(100), y = Dr[:,303], mode = 'lines', name = '3,3'))
fig_D_lines_reshape.show()

In [68]:
D = np.reshape(D,(D.shape[0],D.shape[1]*D.shape[2]))

In [69]:
# Degree centrality (with GPU)
# Make batch indices
batch = dict()
Y = cupy.asarray(D)

# Set up batches
nbatch = 100 # currently recommend 256.
#nbatch = 2
[i,j]=Y.shape
batch_size=np.intc(j/nbatch)
vox_range=np.array([0])
b=0
while vox_range[-1]+1 <= j:
  if vox_range[-1]+1+batch_size > j:
    vox_range=np.arange(vox_range[-1]+1,j+1)
  else:
    vox_range=np.arange(vox_range[-1]+1,vox_range[-1]+1+batch_size)
  batch.update({b:vox_range-1})
  b += 1

DC = np.zeros(j)
linelength = 1

In [70]:
def update_out(bcount,batch):
    out.clear_output()
    with out:
        print('Progress: Batch '+str(bcount)+'/'+str(len(batch)**2))

out = widgets.Output()
out

Output()

In [71]:
# Loop through batches
bcount = 1

for b_col in np.arange(len(batch)):
    # Compute correlations
    R = cupy.zeros([j,batch_size])
    for b_row in np.arange(len(batch)):
        update_out(bcount,batch)
        bcount += 1
        R[batch[b_row],:] = cupy.corrcoef(Y[:,batch[b_row]],Y[:,batch[b_col]],rowvar=False)[len(batch[b_row]):len(batch[b_row])*2,0:len(batch[b_row])].T
    I = np.concatenate((np.zeros([batch_size*(b_col),batch_size]),np.eye(batch_size),np.zeros([batch_size*(nbatch-b_col-1),batch_size])),axis=0)==1
    R[I] = cupy.nan
    DC[batch[b_col]] = cupy.asnumpy(cupy.nanmean(cupy.arctanh(R),axis=0))

In [72]:
eDC = np.reshape(DC,(100,100))
fig1 = px.imshow(eDC,range_color=(0.015,0.05))
#fig1 = px.imshow(eDC)
fig1.show()

In [58]:
eD = np.reshape(np.sum(D,axis=0),(100,100))
#fig_eD = px.imshow(eD,range_color=(50000,100000))
fig_eD = px.imshow(eD)
fig_eD.show()

In [38]:
b_col = 0
R = cupy.zeros([j,batch_size])
for b_row in np.arange(len(batch)):
    R[batch[b_row],:] = cupy.corrcoef(Y[:,batch[b_row]],Y[:,batch[b_col]],rowvar=False)[len(batch[b_row]):len(batch[b_row])*2,0:len(batch[b_row])]

fig1 = px.imshow(cupy.asnumpy(R[0:1000,:]),width=200)
fig1.update_layout(margin=dict(l=0, r=0, t=0, b=0))       
fig1.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fig1 = px.imshow(cupy.asnumpy(R[0:1000,:]),range_color=(0,0.1))             
fig1.show()

In [ ]:
Ri = R[:,30]
eRi = np.reshape(Ri,(100,100))
fig_eRi = px.imshow(cupy.asnumpy(eRi),range_color=(0,0.1))
fig_eRi.show()

In [ ]:
np.save('/home/joshgoh/Desktop/Organoid/Data/derivatives/ses-02/DC',DC)

In [ ]:
DC = np.load('/home/joshgoh/Desktop/Organoid/Data/derivatives/ses-02/DC.npy')

## Widgets

### Header

In [ ]:
Header = Label("EXPLORE SLICE", layout=Layout(display="flex", justify_content="center", width="100%", border='none'))

### Video data selection module

In [ ]:
videopath = widgets.Text(value='',description='Video file full path (e.g. /content/.../video.avi')

vp = widgets.Button(
    description='Read video data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Read video data',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

def vp_click(b):
  #cap = cv2.VideoCapture('/content/drive/Shareddrives/Organoid/Data/ses-02/3.avi')
  cap = cv2.VideoCapture(vp.value)
  frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))
  fc = 0
  ret = True

  while (fc < frameCount  and ret):
    ret, buf[fc] = cap.read()
    fc += 1

  cap.release()
  return buf

buf = vp.on_click(vp_click)

### Time controller module

In [ ]:
st = widgets.IntSlider(value=0,min=0,max=D.shape[1],step=1,description='Timepoint')
def update_fig(t,mini,maxi):
  fig = px.imshow(img[t],range_color=(mini,maxi))
  fig.show()
bt = widgets.Button(
    description='<',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Go back one timepoint',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

ft = widgets.Button(
    description='>',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Go forward one timepoint',
    icon='' # (FontAwesome names without the `fa-` prefix)
)
def bt_click(b):
  with out:
    st.value=st.value-1

def ft_click(b):
  with out:
    st.value=st.value+1
bt.on_click(bt_click)
ft.on_click(ft_click)

### Image intensity range module

In [ ]:
smaxi = widgets.IntSlider(value=np.max(img[0]),min=np.min(img[0]),max=np.max(img[0]),step=1,description='Max intensity',orientation='vertical')
smini = widgets.IntSlider(value=0,min=np.min(img[0]),max=np.max(img[0]),step=1,description='Min intensity',orientation='vertical')

### Degree centrality image module

In [ ]:
dc = widgets.Button(description='Compute degree centrality image')
def = dc_click(b):
  # Compute DC image
  DC = degree_centrality(img)
  # Show DC image
  # Output clusters with coordinates masks
mask = dc.on_click(dc_click)

### Mutual information module

In [ ]:
mi_text = Label("Enter source coordinates", layout=Layout(display="flex", width="30%", border='none'))
xc = widgets.Text(value='',description='x')
yc = widgets.Text(value='',description='y')
rval = widgets.Text(value='',description='Bin size')
nshufval = widgets.Text(value='',description='No. of shuffles')
lagval = widgets.Text(value='',description='Lag')
nbatchval = widgets.Text(value='',description='No. of batches')

compute_mi = widgets.Button(
    description='Compute MI',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Compute MI of this coordinate with image',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

show_mi = widgets.Button(
    description='Show MI',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Show MI image for this coordinate',
    icon='' # (FontAwesome names without the `fa-` prefix)
)
def compute_mi_click(b):
  print('Running mutual information computation for coordinate x: ',xc.value, ', y: ',yc.value)
  MI,R = run_mi(int(xc.value),int(yc.value),int(rval.value),cupy.arange(0,int(lagval.value)),int(nshufval.value),int(nbatchval.value))
  np.save('MI', MI)
  np.save('R.',R)

compute_mi.on_click(compute_mi_click)

# Dashboard

In [ ]:
D = buf[:,:,:,1]
D = np.reshape(D,(D.shape[0],D.shape[1]*D.shape[2])).T
img = buf[:, :, :, 0]

In [ ]:
out = widgets.interactive_output(update_fig, {'t': st, 'mini': smini, 'maxi': smaxi})

widgets.VBox([videopath,
              vp,
              Header,
              widgets.HBox([widgets.VBox([st,
                                          widgets.HBox([bt,
                                                        ft]),
                                          widgets.HBox([smini,
                                                        smaxi]),
                                          mi_text,
                                          widgets.HBox([xc,
                                                        yc]),
                                          rval,
                                          nshufval,
                                          lagval,
                                          nbatchval,
                                          compute_mi]),
                            out])])

In [ ]:
MI = np.load('MI.npy')

#Plot pixels with mutual information to source pixel coordinate.

In [ ]:
#@title
I = np.reshape(MI[:,3],(buf.shape[1],buf.shape[2]))
fig = px.imshow(I,range_color=(0,0.05))
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#@title
fig = make_subplots(rows=3, cols=1)

fig.append_trace(go.Scatter(
    x=x,
    y=buf[:,30,100,1],
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=x,
    y=buf[:,30,100,1],
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=x,
    y=buf[:,30,100,1]
), row=3, col=1)


fig.update_layout(height=600, width=600, title_text="Stacked Subplots")
fig.show()